<a href="https://colab.research.google.com/github/bohyanggg/SC3000_Artificial_Intelligence/blob/main/team_zhby_cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial and Sample Code for Balancing a Pole on a Cart

## Installing dependencies:

In [1]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install gym[classic_control]
!pip install gym[all]
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 70.3 MB/s eta 0:00:00
  Attempting uninstall: pygame
    Found existing installation: pygame 2.3.0
    Uninstalling pygame-2.3.0:
      Successfully uninstalled pygame-2.3.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.0/297.0 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 90.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 KB 45.1 MB/s eta 0:00:00
  Preparing 

## Importing dependencies and define helper functions

In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import RecordVideo
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

## Tutorial: Loading CartPole environment

In [ ]:
# env = gym.make("CartPole-v1")

We can check the action and observation space of this environment. Discrete(2) means that there are two valid discrete actions: 0 & 1.

In [ ]:
# print(env.action_space)

The observation space is given below. The first two arrays define the min and max values of the 4 observed values, corresponding to cart position, velocity and pole angle, angular velocity.

In [ ]:
# print(env.observation_space)

We call each round of the pole-balancing game an "episode". At the start of each episode, make sure the environment is reset, which chooses a random initial state, e.g., pole slightly tilted to the right. This initialization can be achieved by the code below, which returns the observation of the initial state.

In [ ]:
# observation = env.reset()
# print("Initial observations:", observation)

For the CartPole environment, there are two possible actions: 0 for pushing to the left and 1 for pushing to the right. For example, we can push the cart to the left using code below, which returns the new observation, the current reward, an indicator of whether the game ends, and some additional information (not used in this project). For CartPole, the game ends when the pole is significantly tilted or you manage to balance the pole for 500 steps. You get exactly 1 reward for each step before the game ends (i.e., max cumulative reward is 500).

In [ ]:
# observation, reward, done, info = env.step(0)
# print("New observations after choosing action 0:", observation)
# print("Reward for this step:", reward)
# print("Is this round done?", done)

Now we can play a full round of the game using a naive strategy (always choosing action 0), and show the cumulative reward in the round. Note that reward returned by env.step(*) corresponds to the reward for current step. So we have to accumulate the reward for each step. Clearly, the naive strategy performs poorly by surviving only a dozen of steps.

In [ ]:
# observation = env.reset()
# cumulative_reward = 0
# done = False
# while not done:
#     observation, reward, done, info = env.step(0)
#     cumulative_reward += reward
# print("Cumulative reward for this round:", cumulative_reward)

## Task 1: Development of an RL agent

An example of a naive agent is given below, which randomly chooses an action regardless of the observation:

In [ ]:
def rand_policy_agent(observation):
    return random.randint(0, 1)

In [ ]:
import gym
import numpy as np

# define hyperparameters
alpha = 0.1
gamma = 0.99
epsilon = 0.01

# create environment
env = gym.make('CartPole-v1')

# initialize Q-table
n_states = 10**env.observation_space.shape[0]
n_actions = env.action_space.n
q_table = np.zeros((n_states, n_actions))

# define function to discretize observation space
def discretize(observation):
    # create bounds for each feature of observation
    bounds = list(zip(env.observation_space.low, env.observation_space.high))
    # map each feature to a discrete value
    discrete_observation = [int(np.digitize(obs, b)) for obs, b in zip(observation, bounds)]
    # convert discrete observation to single integer
    state = sum([val*(10**i) for i, val in enumerate(discrete_observation)])
    return state

# define function to choose action based on epsilon-greedy policy
def choose_action(state, q_table, epsilon):
    if np.random.uniform() < epsilon:
        action = env.action_space.sample()  # explore
    else:
        action = np.argmax(q_table[state])  # exploit
    return action

# define function to train Q-learning agent
def train_q_learning_agent(env):
    episode_rewards = []
    for i_episode in range(10000):
        state = discretize(env.reset())  # discretize initial observation to get initial state
        done = False
        episode_reward = 0
        while not done:
            action = choose_action(state, q_table, epsilon)
            next_observation, reward, done, info = env.step(action)
            next_state = discretize(next_observation)
            episode_reward += reward
            # update Q-table
            q_table[state, action] = (1 - alpha) * q_table[state, action] + alpha * (reward + gamma * np.max(q_table[next_state]))
            state = next_state
        episode_rewards.append(episode_reward)
        # check if task is solved (total reward for episode is greater than 195)
        if episode_reward > 195:
            break
    return q_table, episode_rewards

# train agent
q_table, episode_rewards = train_q_learning_agent(env)

# plot episode rewards
import matplotlib.pyplot as plt
plt.plot(episode_rewards)
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.title('Q-learning Performance')
plt.show()

NameError: ignored

For Task 1, we can show the observation and chosen action below:

In [ ]:
# observation = env.reset()
# action = rand_policy_agent(observation)
# print("Observation:", observation)
# print("Chosen action:", action)

## Task 2: Demonstrate the effectiveness of the RL agent

For this task, use the agent developed in Task 1 to play the game for 100 episodes (refer to tutorial for how to play a round), record the cumulative reward for each round, and plot the reward for each round. A sample plotting code is given below. Note that you must include code to play for 100 episodes and use the code to obtain round_results for plotting. DO NOT record the round results in advance and paste the results to the notebook.

In [ ]:
# episode_results = np.random.randint(150, 2500, size=100)
# plt.plot(episode_results)
# plt.title('Cumulative reward for each episode')
# plt.ylabel('Cumulative reward')
# plt.xlabel('episode')
# plt.show()

Print the average reward over the 100 episodes.

In [ ]:
# print("Average cumulative reward:", episode_results.mean())
# print("Is my agent good enough?", episode_results.mean() > 195)

## Task 3: Render one episode played by the agent

Plug your agent to the code below to obtain rendered result.

In [ ]:
# env = RecordVideo(gym.make("CartPole-v1"), "./video")
# observation = env.reset()
# while True:
#     env.render()
#     #your agent goes here
#     action = rand_policy_agent(observation)
#     #action = train_q_learning_agent(env)
#     observation, reward, done, info = env.step(action) 
#     if done: 
#       break;    
# env.close()
# show_video()

# render one episode played by the agent using RecordVideo function
video_env = RecordVideo(env, "./video.mp4")
observation = video_env.reset()
done = False
total_reward = 0
while not done:
    video_env.render()
    action = np.argmax(q_table[discretize(observation)])
    observation, reward, done, info = video_env.step(action)
    total_reward += reward

# close environment and show video
video_env.close()
show_video()